In [1]:
import pandas as pd
import numpy as np

C:\Users\Dell\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df= pd.read_csv('IMDB Dataset.csv')

In [3]:
df['review'][0] # this is gonna need a lot of preprocessing

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [6]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary nltk resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"\W", " ", text)  # Remove punctuation
    words = word_tokenize(text)  # Tokenize (split into words)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Remove stopwords & lemmatize
    return " ".join(words)

df["cleaned_review"] = df["review"].apply(preprocess_text)  # Apply function to each row
print(df[["review", "cleaned_review"]].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  one reviewer mentioned watching 1 oz episode h...  
1  wonderful little production filming technique ...  
2  thought wonderful way spend time hot summer we...  
3  basically family little boy jake think zombie ...  
4  petter mattei love time money visually stunnin...  


In [8]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
len(df.iloc[0]['review'])

1761

In [10]:
len(df.iloc[0]['cleaned_review'])

1072

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Use 5000 most frequent words
X = vectorizer.fit_transform(df["cleaned_review"]).toarray()  # Convert text into numbers
y = df["sentiment"].map({"positive": 1, "negative": 0})  # Convert labels to 1 (positive) and 0 (negative)


In [12]:
X.shape

(50000, 5000)

In [13]:
y.shape

(50000,)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8881
              precision    recall  f1-score   support

           0       0.90      0.87      0.89      4961
           1       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [15]:
new_reviews = ["This movie was amazing! The story was great.", "I hated this film. It was boring and slow."]
new_reviews_cleaned = [preprocess_text(review) for review in new_reviews]
new_reviews_vectorized = vectorizer.transform(new_reviews_cleaned).toarray()
predictions = model.predict(new_reviews_vectorized)

In [16]:
predictions

array([1, 0], dtype=int64)

In [17]:
from sklearn.ensemble import RandomForestClassifier

# Split dataset into training and testing sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model_rf = RandomForestClassifier()
model_rf.fit(X_train_rf, y_train_rf)

# Predictions
y_pred_rf = model.predict(X_test_rf)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test_rf, y_pred_rf))
print(classification_report(y_test_rf, y_pred_rf))

Accuracy: 0.8881
              precision    recall  f1-score   support

           0       0.90      0.87      0.89      4961
           1       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [19]:
from sklearn import svm

In [20]:


# Split dataset into training and testing sets
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model_rf_svm = RandomForestClassifier()
model_rf_svm.fit(X_train_rf, y_train_rf)

# Predictions
y_pred_svm = model.predict(X_test_rf)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test_svm, y_pred_svm))
print(classification_report(y_test_svm, y_pred_svm))


Accuracy: 0.8881
              precision    recall  f1-score   support

           0       0.90      0.87      0.89      4961
           1       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [18]:
# from sklearn.model_selection import RandomizedSearchCV

# param_dist = {
    'n_estimators': [100, 200, 300, 500],  # Number of trees in the forest
    'max_depth': [10, 20, 30, None],  # Max depth of trees
    'min_samples_split': [2, 5, 10, 15],  # Minimum samples to split
    'min_samples_leaf': [1, 2, 5],  # Minimum samples at leaf node
    'max_features': ['auto', 'sqrt', 'log2', 10],  # Features per split
    'bootstrap': [True, False],  # Whether to use bootstrap samples
    'criterion': ['gini', 'entropy']  # Split quality measure
}

# rf_model = RandomForestClassifier(random_state=42)

# Step 3: Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(
    rf_model, param_distributions=param_dist, n_iter=10,  # 10 iterations
    cv=3, n_jobs=-1, random_state=42, verbose=2
)

# random_search.fit(X_train_rf, y_train_rf)

IndentationError: unexpected indent (4268182710.py, line 4)